# Loan Default Prediction: A Neural Network Approach
22 July 2020

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Matplotlib
plt.rcParams['figure.figsize'] = 9,6
plt.rcParams.update({'font.size': 16})

# Seaborn
sns.set_style('whitegrid')
sns.set_palette("PuBuGn_d")

### Data

In [2]:
# Set Data Directory
data_dir = '../Data_Hands-on/'

# Read Files
X_train = pd.read_csv(data_dir + 'X_train.csv')
X_test  = pd.read_csv(data_dir + 'X_test.csv' )
y_train = pd.read_csv(data_dir + 'y_train.csv')
y_test  = pd.read_csv(data_dir + 'y_test.csv')

In [6]:
X_train

,loan_amnt,funded_amnt,term,int_rate,installment,annual_inc,dti,delinq_2yrs,mths_since_last_delinq,open_acc,...,total_pymnt,total_rec_late_fee,last_pymnt_amnt,mths_since_last_major_derog,emp_length_num,grade_num,home_ownership_MORTGAGE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
0,25000,25000,36,14.09,855.54,153000.0,10.13,0.0,58.0,8.0,...,27851.60,0.0,19303.40,68.0,11,5,1,0,0,0
1,10000,10000,36,17.77,360.38,74000.0,13.20,1.0,23.0,13.0,...,10434.80,0.0,9714.02,23.0,3,3,0,0,0,1
2,10000,10000,36,15.31,348.18,54600.0,22.77,0.0,NaN,23.0,...,10939.90,0.0,8504.78,NaN,5,4,1,0,0,0
3,30000,30000,60,16.99,745.42,110000.0,10.31,0.0,NaN,9.0,...,745.28,0.0,745.42,NaN,8,3,0,0,1,0
4,23500,23500,60,15.61,566.62,62000.0,17.23,0.0,NaN,7.0,...,25282.40,0.0,22449.30,NaN,11,4,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,16100,16100,60,25.89,481.00,45000.0,7.84,0.0,NaN,4.0,...,18136.90,0.0,15735.70,NaN,2,0,0,0,0,1
49996,24000,24000,36,9.49,768.68,115000.0,20.23,1.0,6.0,17.0,...,24374.40,0.0,23618.40,NaN,11,5,0,0,0,1
49997,20000,20000,36,13.92,682.74,84996.0,12.61,0.0,NaN,21.0,...,20978.30,0.0,519.79,NaN,9,4,0,0,0,1
49998,3700,3700,60,16.24,90.45,89000.0,21.63,0.0,NaN,22.0,...,4311.72,0.0,3226.40,NaN,4,4,0,0,1,0
